In [1]:
# imports and libs
!pip3 install transformers
!pip3 install trl
!pip3 install bitsandbytes
!pip3 install datasets
!pip3 install peft

import torch
from transformers import AutoModelForCausalLM, DataCollatorWithPadding
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BitsAndBytesConfig
from torch.optim import AdamW
from datasets import load_dataset
from torch.utils.data import DataLoader
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, TaskType
from transformers import Trainer, TrainingArguments
from trl import SFTConfig, SFTTrainer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
checkpoint = "microsoft/Phi-3-mini-4k-instruct"
checkpoint2 = "bert-base-uncased"

bnb_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.float32
)

# Load model directly
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer2 = AutoTokenizer.from_pretrained(checkpoint2)

optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [3]:
device = torch.device("cpu")

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")

model.to(device)
print("Model loaded to device:", device)

Model loaded to device: cuda


In [4]:
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,                   # the rank of the adapter, the lower the fewer parameters you'll need to train
    lora_alpha=16,         # multiplier, usually 2*r
    bias="none",           # BEWARE: training biases *modifies* base model's behavior
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
    # Newer models, such as Phi-3 at time of writing, may require
    # manually setting target modules
    target_modules=['o_proj', 'qkv_proj', 'gate_up_proj', 'down_proj'],
)
# config = LoraConfig(
#     task_type=TaskType.CAUSAL_LM, # type of task to train on
#     inference_mode=False, # set to False for training
#     r=8, # dimension of the smaller matrices
#     lora_alpha=32, # scaling factor
#     lora_dropout=0.1 # dropout of LoRA layers
# )
# model.add_adapter(config, adapter_name="lora")
model = get_peft_model(model, config)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Phi3ForCausalLM(
      (model): Phi3Model(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (layers): ModuleList(
          (0-31): 32 x Phi3DecoderLayer(
            (self_attn): Phi3Attention(
              (o_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (

In [5]:
# trainable_parms, tot_parms = model.get_nb_trainable_parameters()
# print(f'Trainable parameters:             {trainable_parms/1e6:.2f}M')
# print(f'Total parameters:                 {tot_parms/1e6:.2f}M')
# print(f'Fraction of trainable parameters: {100*trainable_parms/tot_parms:.2f}%')

In [6]:
def tokenize(example):
    return tokenizer(example["input"], example["output"], truncation=True)

def tokenize2(example):
    return tokenizer2(example["sentence1"], example["sentence2"], truncation=True)

health_dataset = load_dataset("lavita/ChatDoctor-HealthCareMagic-100k", split="train")
# health_dataset = health_dataset[:5000]
health_dataset = health_dataset.take(5000)
health_dataset

# health_dataset[0]

# yelp_dataset = load_dataset("yelp_review_full")
# glue_dataset = load_dataset("glue", "mrpc")

# health_dataset = health_dataset.map(tokenize, batched=True)
# glue_dataset = glue_dataset.map(tokenize2, batched=True)

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# data_collator2 = DataCollatorWithPadding(tokenizer=tokenizer2)

README.md:   0%|          | 0.00/542 [00:00<?, ?B/s]

(…)-00000-of-00001-5e7cb295b9cff0bf.parquet:   0%|          | 0.00/70.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/112165 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 5000
})

In [7]:
# print(health_dataset['train'][:2])
print(health_dataset.column_names)
# print(yelp_dataset.column_names)
# print(glue_dataset.column_names)

['instruction', 'input', 'output']


In [8]:
# only run once
# health_dataset = health_dataset.remove_columns(["input", "output"])
# health_dataset = health_dataset.remove_columns(["instruction"])
# yelp_dataset = yelp_dataset.remove_columns(["text"])
# yelp_dataset = yelp_dataset.rename_column("label", "labels")
# glue_dataset = glue_dataset.remove_columns(["sentence1", "sentence2", "idx"])
# glue_dataset = glue_dataset.rename_column("label", "labels")

if "input" in health_dataset.column_names:
    health_dataset = health_dataset.rename_column("input", "prompt")
if "output" in health_dataset.column_names:
    health_dataset = health_dataset.rename_column("output", "completion")
if "instruction" in health_dataset.column_names:
    health_dataset = health_dataset.remove_columns(["instruction"])

health_dataset.set_format(type="torch")
health_dataset
# glue_dataset.set_format(type="torch")
# print(glue_dataset["train"].column_names)

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 5000
})

In [9]:
def format_dataset(examples):
    if isinstance(examples["prompt"], list):
        output_texts = []
        for i in range(len(examples["prompt"])):
            converted_sample = [
                {"role": "user", "content": examples["prompt"][i]},
                {"role": "assistant", "content": examples["completion"][i]},
            ]
            output_texts.append(converted_sample)
        return {'messages': output_texts}
    else:
        converted_sample = [
            {"role": "user", "content": examples["prompt"]},
            {"role": "assistant", "content": examples["completion"]},
        ]
        return {'messages': converted_sample}

if "prompt" in health_dataset.column_names:
    health_dataset = health_dataset.map(format_dataset, batched=True).remove_columns(["prompt", "completion"])
health_dataset

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset({
    features: ['messages'],
    num_rows: 5000
})

In [10]:
# training_args = TrainingArguments(
#     output_dir="health_assistant_model",
#     eval_strategy="epoch"
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=health_dataset["train"],
# )


In [11]:
print(tokenizer.chat_template)

{% for message in messages %}{% if message['role'] == 'system' %}{{'<|system|>
' + message['content'] + '<|end|>
'}}{% elif message['role'] == 'user' %}{{'<|user|>
' + message['content'] + '<|end|>
'}}{% elif message['role'] == 'assistant' %}{{'<|assistant|>
' + message['content'] + '<|end|>
'}}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>
' }}{% else %}{{ eos_token }}{% endif %}


In [12]:
sft_config = SFTConfig(
    ## GROUP 1: Memory usage
    # These arguments will squeeze the most out of your GPU's RAM
    # Checkpointing
    gradient_checkpointing=True,
    # this saves a LOT of memory
    # Set this to avoid exceptions in newer versions of PyTorch
    gradient_checkpointing_kwargs={'use_reentrant': False},
    # Gradient Accumulation / Batch size
    # Actual batch (for updating) is same (1x) as micro-batch size
    gradient_accumulation_steps=1,
    # The initial (micro) batch size to start off with
    per_device_train_batch_size=16,
    # If batch size would cause OOM, halves its size until it works
    auto_find_batch_size=True,

    ## GROUP 2: Dataset-related
    max_seq_length=64,
    # Dataset
    # packing a dataset means no padding is needed
    packing=True,

    ## GROUP 3: These are typical training parameters
    num_train_epochs=10,
    learning_rate=0.001,
    # Optimizer
    # 8-bit Adam optimizer - doesn't help much if you're using LoRA!
    optim='paged_adamw_8bit',

    ## GROUP 4: Logging parameters
    logging_steps=10,
    logging_dir='./logs',
    output_dir='./health_assistant_adapter',
    report_to='none'
)

In [13]:
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    args=sft_config,
    train_dataset=health_dataset,
)

Converting train dataset to ChatML:   0%|          | 0/5000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [14]:
dl = trainer.get_train_dataloader()
batch = next(iter(dl))

In [15]:
batch['input_ids'][0], batch['labels'][0]

(tensor([29898,  2052,   307, 29916, 29889, 29871, 29955, 29899, 29896, 29900,
          3841,   297,   263,  4098, 20149, 29123,   304,   445,  1108, 29892,
           474,  4459, 27719, 29892,  1161,   420, 29892,   316,  2590, 29892,
         14919, 21549, 29892,   322,  3458,   766,  2098,  1108,   636,   578,
           715, 29920,   565,  5019,  4368,   592, 29991,  1128,   304,   474,
          1369,   590, 11785,  2834,  1728,  1438,  1108, 29892,   769,   372,
           674,   367,  1371,   363], device='cuda:0'),
 tensor([29898,  2052,   307, 29916, 29889, 29871, 29955, 29899, 29896, 29900,
          3841,   297,   263,  4098, 20149, 29123,   304,   445,  1108, 29892,
           474,  4459, 27719, 29892,  1161,   420, 29892,   316,  2590, 29892,
         14919, 21549, 29892,   322,  3458,   766,  2098,  1108,   636,   578,
           715, 29920,   565,  5019,  4368,   592, 29991,  1128,   304,   474,
          1369,   590, 11785,  2834,  1728,  1438,  1108, 29892,   769,   3

In [16]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,2.916000
20,2.686400
30,2.722600
40,2.647100
50,2.754600
60,2.702200
70,2.646300
80,2.722300
90,2.638300
100,2.639400


TrainOutput(global_step=12960, training_loss=1.5320013247890236, metrics={'train_runtime': 21066.8056, 'train_samples_per_second': 9.842, 'train_steps_per_second': 0.615, 'total_flos': 2.974025513631744e+17, 'train_loss': 1.5320013247890236})

In [17]:
def gen_prompt(tokenizer, sentence):
    converted_sample = [
        {"role": "user", "content": sentence},
    ]
    prompt = tokenizer.apply_chat_template(converted_sample,
                                           tokenize=False,
                                           add_generation_prompt=True)
    return prompt

In [18]:
sentence = 'The Force is strong in you!'
prompt = gen_prompt(tokenizer, sentence)
print(prompt)

<|user|>
The Force is strong in you!<|end|>
<|assistant|>



In [19]:
def generate(model, tokenizer, prompt, max_new_tokens=64, skip_special_tokens=False):
    tokenized_input = tokenizer(prompt, add_special_tokens=False, return_tensors="pt").to(model.device)

    model.eval()
    generation_output = model.generate(**tokenized_input,
                                       eos_token_id=tokenizer.eos_token_id,
                                       max_new_tokens=max_new_tokens)

    output = tokenizer.batch_decode(generation_output,
                                    skip_special_tokens=skip_special_tokens)
    return output[0]

In [21]:
print(generate(model, tokenizer, prompt))

<|user|> The Force is strong in you!<|end|><|assistant|> Hello, Welcome to Chat Doctor, The present situation is called as spasm and is a common problem in persons who are taking a long treatment with a painkiller. The painkiller itself can cause the problem but more common cause is the patient has a pathology in the underlying muscles which is causing sp


In [22]:
print(generate(model, tokenizer, gen_prompt(tokenizer, input("Enter something to ask doctor:"))))

Enter something to ask doctor:I am getting headaches and dizzy.
<|user|> I am getting headaches and dizzy.<|end|><|assistant|> Hello, I have studied your case. Ibuprofen may cause dizziness.  You should consult your doctor and take treatment accordingly.  Sometimes vitamin B 12 deficiency can cause dizziness.  You should take B 12 supplement.  Consult your doctor and take


In [23]:
trainer.save_model('local-phi3-mini-healthcare-provider')

In [ ]:
health_dataloader = DataLoader(health_dataset["train"], batch_size=8, shuffle=True, collate_fn=data_collator)
yelp_dataloader = DataLoader(yelp_dataset["train"], batch_size=8, shuffle=True)
glue_dataloader = DataLoader(glue_dataset["train"], batch_size=8, shuffle=True, collate_fn=data_collator2)

In [ ]:
print(model.get_memory_footprint()/1e6)

In [ ]:
print(model.num_parameters())

In [ ]:
for batch in health_dataloader:
    # for k, v in batch.items():
        # print(k, v)
    # outputs = model(**batch)
    break
{k: v.shape for k, v in batch.items()}

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "How are you?"},
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
chat_input = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate response
chat_outputs = model.generate(**chat_input, max_new_tokens=50)
response = tokenizer.decode(chat_outputs[0][chat_input['input_ids'].shape[-1]:], skip_special_tokens=True) # Decode only the response part
print("\nAssistant Response:", response)